In [169]:
# create feature 'after' for probability matrix
# create features 'entrytime' - 'exittime'
#calculate transition probabilities (5x5 matrix)

In [270]:
import pandas as pd 

#TO DO:
# merge data
# new column costumer

mo = pd.read_csv('/Users/valerie/Desktop/data_project_8/monday.csv',sep=';')
tu = pd.read_csv('/Users/valerie/Desktop/data_project_8/tuesday.csv',sep=';')
we = pd.read_csv('/Users/valerie/Desktop/data_project_8/wednesday.csv',sep=';')
thu = pd.read_csv('/Users/valerie/Desktop/data_project_8/thursday.csv',sep=';')
fr = pd.read_csv('/Users/valerie/Desktop/data_project_8/friday.csv',sep=';')

In [271]:
#differentiate between different datasets, explanation further down
mo['weekday'] = 1000
tu['weekday'] = 2000
we['weekday'] = 3000
thu['weekday'] = 4000
fr['weekday'] = 5000


In [272]:
# concatenate datasets
pieces = (mo,tu,we,thu,fr)
df = pd.concat(pieces)
df.shape

In [275]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y-%m-%d %H:%M:%S")
df['entrytime'] = df['timestamp']
df.set_index('timestamp', inplace=True)


In [276]:
df.head()

,customer_no,location,weekday,entrytime
timestamp,,,,
2019-09-02 07:03:00,1,dairy,1000,2019-09-02 07:03:00
2019-09-02 07:03:00,2,dairy,1000,2019-09-02 07:03:00
2019-09-02 07:04:00,3,dairy,1000,2019-09-02 07:04:00
2019-09-02 07:04:00,4,dairy,1000,2019-09-02 07:04:00
2019-09-02 07:04:00,5,spices,1000,2019-09-02 07:04:00


In [277]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24877 entries, 2019-09-02 07:03:00 to 2019-09-06 21:50:00
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_no  24877 non-null  int64         
 1   location     24877 non-null  object        
 2   weekday      24877 non-null  int64         
 3   entrytime    24877 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 971.8+ KB


In [278]:
# problem: for every day of the week costumers start counting at 1 again. It would be false to merge the datasets and group the costumers like this, according to Kristian. To avoid this I create a new costumer feature
# where I add numbers in the steps of 1000 to each costumer number. Customer_no 1 for monday will have the number 1001, Customer_no 1 for tuesday will have the number 2001 etc. As there are ca. 1500 costumers in every dataset there won't be any doubles, I think.
# I'm open to other solutions!
df['customer'] = df['customer_no'] + df['weekday']
df.head()

,customer_no,location,weekday,entrytime,customer
timestamp,,,,,
2019-09-02 07:03:00,1,dairy,1000,2019-09-02 07:03:00,1001
2019-09-02 07:03:00,2,dairy,1000,2019-09-02 07:03:00,1002
2019-09-02 07:04:00,3,dairy,1000,2019-09-02 07:04:00,1003
2019-09-02 07:04:00,4,dairy,1000,2019-09-02 07:04:00,1004
2019-09-02 07:04:00,5,spices,1000,2019-09-02 07:04:00,1005


In [283]:
# groupby, resample and forwardfill
df_prob = df.groupby('customer').resample('1Min').ffill()

df_prob.head()


customer_no  location  weekday  \
customer timestamp                                             
1001     2019-09-02 07:03:00            1     dairy     1000   
         2019-09-02 07:04:00            1     dairy     1000   
         2019-09-02 07:05:00            1  checkout     1000   
1002     2019-09-02 07:03:00            2     dairy     1000   
         2019-09-02 07:04:00            2     dairy     1000   

                                       entrytime  customer  
customer timestamp                                          
1001     2019-09-02 07:03:00 2019-09-02 07:03:00      1001  
         2019-09-02 07:04:00 2019-09-02 07:03:00      1001  
         2019-09-02 07:05:00 2019-09-02 07:05:00      1001  
1002     2019-09-02 07:03:00 2019-09-02 07:03:00      1002  
         2019-09-02 07:04:00 2019-09-02 07:03:00      1002

In [284]:
#shift() time lagged feature, create before - after
df_prob['after'] = df_prob['location'].shift(-1)

In [285]:
#shift() time lagged feature, create entrytime - exittime
df_prob['exittime'] = df_prob['entrytime'].shift(-1)

In [290]:
#optional!
#df_prob = df_prob.drop('customer_no', 1)
#df_prob = df_prob.rename(columns={'location': 'before'})
df_prob = df_prob.drop('customer', 1)

In [291]:
# drop location == 'checkout', because here we have the switch from one costumer to another! the values we find here in feature 'after' and 'exittime' are not correct
delete = df_prob[ (df_prob['location'] == 'checkout')].index
df_prob.drop(delete, inplace=True)

In [292]:
df_prob.head(10)

customer_no location  weekday  \
customer timestamp                                            
1001     2019-09-02 07:03:00            1    dairy     1000   
         2019-09-02 07:04:00            1    dairy     1000   
1002     2019-09-02 07:03:00            2    dairy     1000   
         2019-09-02 07:04:00            2    dairy     1000   
         2019-09-02 07:05:00            2    dairy     1000   
1003     2019-09-02 07:04:00            3    dairy     1000   
         2019-09-02 07:05:00            3    dairy     1000   
1004     2019-09-02 07:04:00            4    dairy     1000   
         2019-09-02 07:05:00            4    dairy     1000   
         2019-09-02 07:06:00            4    dairy     1000   

                                       entrytime     after            exittime  
customer timestamp                                                              
1001     2019-09-02 07:03:00 2019-09-02 07:03:00     dairy 2019-09-02 07:03:00  
         2019-09-02 07:04:00 2019-09-02 07:03:00  checkout 2019-09-02 07:05:00  
1002     2019-09-02 07:03:00 2019-09-02 07:03:00     dairy 2019-09-02 07:03:00  
         2019-09-02 07:04:00 2019-09-02 07:03:00     dairy 2019-09-02 07:03:00  
         2019-09-02 07:05:00 2019-09-02 07:03:00  checkout 2019-09-02 07:06:00  
1003     2019-09-02 07:04:00 2019-09-02 07:04:00     dairy 2019-09-02 07:04:00  
         2019-09-02 07:05:00 2019-09-02 07:04:00  checkout 2019-09-02 07:06:00  
1004     2019-09-02 07:04:00 2019-09-02 07:04:00     dairy 2019-09-02 07:04:00  
         2019-09-02 07:05:00 2019-09-02 07:04:00     dairy 2019-09-02 07:04:00  
         2019-09-02 07:06:00 2019-09-02 07:04:00     dairy 2019-09-02 07:04:00

In [289]:
# 5X5 probability matrix
#You can sum up the counts and calculate probabilities comfortably with pd.crosstab():

pd.crosstab(df_prob['after'], df_prob['location'], normalize=0)

location,dairy,drinks,fruit,spices
after,,,,
checkout,0.246056,0.282594,0.344479,0.126871
dairy,0.898363,0.004243,0.048797,0.048597
drinks,0.121374,0.678043,0.081305,0.119278
fruit,0.059089,0.056832,0.846036,0.038043
spices,0.122055,0.112738,0.085851,0.679356


In [ ]:
# note: it's not a 5x5 matrix  anymore because we dropped location == checkout...

In [ ]:
LABELS =  ['checkout', 'dairy', 'drinks', 'fruit', 'spices']

prob = {

'entrance': [0.0, 0.25, 0.25, 0.25, 0.25]

'dairy' : [0.24,0.89,0.12,0.05,0.12]
'drinks': [0.28,0.00,0.67,0.05,0.11]
'fruit': [0.34,0.04,0.08,0.84,0.08]
'spices': [0.12,0.04,0.11,0.03,0.67]

'checkout': [1.0, 0.0, 0.0, 0.0, 0.0]

}

## Monte Carlo Markov Chain Simulation

In [9]:
# code from flight_simulator.py

from random import choices


STATES = [
'checkout', 'dairy', 'drinks', 'fruit', 'spices'
]

def mcmc(initial_state, P):
    """
    runs a Monte Carlo Markov Chain until at checkout
    
    :params:

    :initial_state: this is a string with the state name that you start with
    :P: a dict of lists with the transition probability matrix
    """
    history = [initial_state]
    state = initial_state
    while state != 'checkout':
        probs = P[state]
        state = choices(STATES, probs) [0]
        history.append(state)
        if history[-1] == 'checkout':
            return history

P = {


'entrance': [0.0, 0.25, 0.25, 0.25, 0.25],

'dairy' : [0.24,0.89,0.12,0.05,0.12],
'drinks': [0.28,0.00,0.67,0.05,0.11],
'fruit': [0.34,0.04,0.08,0.84,0.08],
'spices': [0.12,0.04,0.11,0.03,0.67],

'checkout': [1.0, 0.0, 0.0, 0.0, 0.0]


}
print(f"at checkout after shopping through {len(mcmc('entrance', P))} locations")

at checkout after shopping through 9 locations
